In [1]:
'''
In this file 
We do data processing for SketchVAE training
Since it requires rhythm and pitch tokens, we should process the previous data into the rhythm and pitch data first.
'''
from loader.dataloader import DataLoader
import numpy as np

data_path = [
    "data/irish_train.npy", 
    "data/irish_validate.npy", 
    "data/irish_test.npy"
]

In [2]:
# import data
train_x = np.load(data_path[0],allow_pickle = True)
validate_x = np.load(data_path[1],allow_pickle = True)
test_x = np.load(data_path[2],allow_pickle = True)


In [3]:
# note extraction
hold_state = 128
rest_state = 129
def extract_note(x, pad_token = 128):
    d = []
    for i in x:
        if i < 128:
            d.append(i)
    ori_d = len(d)
    d.extend([pad_token] * (len(x) - len(d)))
    return np.array(d), ori_d

def extract_rhythm(x, hold_token = 2, rest_token = 3):
    d = []
    for i in x:
        if i < 128:
             d.append(1)
        elif i == hold_state:
             d.append(hold_token)
        else:
             d.append(rest_token)
    return np.array(d)
             

In [9]:
# process rhythm and pitch tokens
split_size = 24
new_data = []
# change here to be train_x/validate_x/test_x
for i,d in enumerate(validate_x):
    d = np.array(d["notes"])
    ds = np.split(d, list(range(split_size,len(d),split_size)))
    data = []
    for sd in ds:
        if len(sd) != split_size:
            continue
        q,k = extract_note(sd)
        if k == 0:
            continue
        s = extract_rhythm(sd)
        data.append([sd,q,s,k])
    new_data.append(data)
    if i % 1000 == 0:
        print("processed:", i)

processed: 0
processed: 1000
processed: 2000
processed: 3000
processed: 4000


In [10]:
# now you can see what processed data looks like [gd, pitch, rhythm, pitch_len]
print(new_data[0][0])

[array([ 60, 128, 128, 128, 128, 128,  64, 128, 128,  60, 128, 128,  65,
       128, 128,  62, 128, 128,  64, 128, 128,  62, 128, 128]), array([ 60,  64,  60,  65,  62,  64,  62, 128, 128, 128, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128]), array([1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1,
       2, 2]), 7]


In [11]:
# we need to extract each measure in each song
final_data = []
for d in new_data:
    for dd in d:
        final_data.append(dd)
print(len(final_data))

143156


In [12]:
# save the data named: irish_xxx_chord_rhythm.npy
final_np = np.array(final_data, dtype=object)
np.save("data/irish_validate_chord_rhythm.npy", final_data)